In [1]:
import tensorflow as tf
import numpy as np
import pandas as pd
from glob import glob
from tensorflow.python.framework import ops
from tensorflow.python.framework import dtypes
from sklearn.preprocessing import OneHotEncoder
from tensorflow.python.framework import graph_util
from os.path import isfile

In [2]:
data_directory = 'data/'
image_directory = 'data/Pre_train/'

In [3]:
ohe = OneHotEncoder()

In [4]:
ohe.fit(np.arange(3).reshape(-1, 1))

OneHotEncoder(categorical_features='all', dtype=<class 'numpy.float64'>,
       handle_unknown='error', n_values='auto', sparse=True)

In [5]:
ohe.transform([[0], [1], [2]]).toarray()

array([[ 1.,  0.,  0.],
       [ 0.,  1.,  0.],
       [ 0.,  0.,  1.]])

In [6]:
size = 32
batch_size = 256
num_channels = 3
training_iters = 1001
display = 10
drop = 0.75
save_step = 50

In [7]:
def read_label_files(csv_file):
    file = open(data_directory+csv_file, 'r')
    filepaths = []
    labels = []
    for i in file:
        filename, label = i.split(',')
        filepaths.append(image_directory+filename)
        labels.append(int(label))
    return filepaths, labels    

In [8]:
train_path, train_labels = read_label_files('train_set.csv')
vali_path, vali_labels = read_label_files('vali_set.csv')

In [9]:
vali_batch_size = len(vali_labels)

In [10]:
train_path_tensor = ops.convert_to_tensor(train_path, dtype=dtypes.string)
vali_path_tensor = ops.convert_to_tensor(vali_path, dtype=dtypes.string)
train_labels_tensor = ops.convert_to_tensor(train_labels, dtype=dtypes.int32)
vali_labels_tensor = ops.convert_to_tensor(vali_labels, dtype=dtypes.int32)

In [11]:
train_input_queue = tf.train.slice_input_producer(
                                    [train_path_tensor, train_labels_tensor],
                                    shuffle=True)
vali_input_queue = tf.train.slice_input_producer(
                                    [vali_path_tensor, vali_labels_tensor],
                                    shuffle=True)

In [12]:
file_content = tf.read_file(train_input_queue[0])
train_image = tf.image.decode_jpeg(file_content, channels=num_channels)
train_image /= 255
train_label = train_input_queue[1]

file_content = tf.read_file(vali_input_queue[0])
vali_image = tf.image.decode_jpeg(file_content, channels=num_channels)
vali_image /= 255
vali_label = vali_input_queue[1]

In [13]:
train_image.set_shape([size, size, num_channels])
vali_image.set_shape([size, size, num_channels])

train_batch = tf.train.batch(
                            [train_image, train_label],
                            batch_size= batch_size
                            )
vali_batch = tf.train.batch(
                            [vali_image, vali_label],
                            batch_size = vali_batch_size
                            )

In [14]:
def freeze_graph(model_folder):

    checkpoint = tf.train.get_checkpoint_state(model_folder)
    input_checkpoint = checkpoint.model_checkpoint_path
    model_name = 'model_v3'
    absolute_model_folder = "/".join(input_checkpoint.split('/')[:-1])
    output_graph = absolute_model_folder + "/" + model_name + ".pb"

    output_node_names = "final_output"

    clear_devices = True

    saver = tf.train.import_meta_graph(input_checkpoint + '.meta', clear_devices=clear_devices)

    graph = tf.get_default_graph()
    input_graph_def = graph.as_graph_def()

    with tf.Session() as sess:
        saver.restore(sess, input_checkpoint)

        output_graph_def = graph_util.convert_variables_to_constants(
            sess,
            input_graph_def,
            output_node_names.split(",")
        )

        with tf.gfile.GFile(output_graph, "wb") as f:
            f.write(output_graph_def.SerializeToString())
        print("%d ops in the final graph." % len(output_graph_def.node))


In [15]:
# with tf.Session() as sess:
#     sess.run(tf.global_variables_initializer())
#     coord = tf.train.Coordinator()
#     threads = tf.train.start_queue_runners(coord=coord)
    
#     for i in range(1):
#         test = sess.run(train_batch)
#     coord.request_stop()
#     coord.join(threads)

In [16]:
# image, label = test

In [17]:
# print(image.shape)
# print(label.shape)

In [18]:
X = tf.placeholder(tf.float32, shape = [None, size, size, 3])
keep_prob = tf.placeholder(tf.float32)
y = tf.placeholder(tf.float32, shape=[None, 3])

In [19]:
weights = {
        'wc1':tf.Variable(tf.random_normal([5, 5, 3, 32])),
        'wc2':tf.Variable(tf.random_normal([5, 5, 32, 64])),
        'wc3':tf.Variable(tf.random_normal([3, 3, 64, 64])),
        'wf1':tf.Variable(tf.random_normal([8 * 8 * 64, 1024])),
        'wf2':tf.Variable(tf.random_normal([1024, 100])),
        'out':tf.Variable(tf.random_normal([100, 3]))
         }

In [20]:
bias = {
        'bc1' : tf.Variable(tf.random_normal([32])),
        'bc2' : tf.Variable(tf.random_normal([64])),
        'bc3' : tf.Variable(tf.random_normal([64])),
        'bf1' : tf.Variable(tf.random_normal([1024])),
        'bf2' : tf.Variable(tf.random_normal([100])),
        'out':tf.Variable(tf.random_normal([3]))
        }

In [21]:
def conv_net(input_image):
        conv1 = tf.nn.conv2d(input_image, weights['wc1'], [1, 1, 1, 1], padding='SAME')
        conv1 = tf.add(conv1, bias['bc1'])
        conv1 = tf.nn.relu(conv1)
        max1 = tf.nn.max_pool(conv1, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv2 = tf.nn.conv2d(max1, weights['wc2'], [1, 1, 1, 1], padding='SAME')
        conv2 = tf.add(conv2, bias['bc2'])
        conv2 = tf.nn.relu(conv2)
        max2 = tf.nn.max_pool(conv2, [1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
        conv3 = tf.nn.conv2d(max2, weights['wc3'], [1, 1, 1, 1], padding='SAME')
        conv3 = tf.add(conv3, bias['bc3'])
        conv3 = tf.nn.relu(conv3)
        b, h, w, c = conv3.get_shape().as_list()
        unrolled = tf.reshape(max2, [-1, h * w * c])
        full1 = tf.add(tf.matmul(unrolled, weights['wf1']), bias['bf1'])
        full1 = tf.nn.relu(full1)
        full2 = tf.add(tf.matmul(full1, weights['wf2']), bias['bf2'])
        full2 = tf.nn.relu(full2)
        out = tf.add(tf.matmul(full2, weights['out']), bias['out'])
        return out

In [ ]:
cifar_weights = {
    'wc1' : tf.Variable(tf.truncated_normal([5, 5, 3, 96])),
    'wc2' : tf.Variable(tf.truncated_normal([1, 1, 96, 96])),
    'wc3' : tf.Variable(tf.truncated_normal([5, 5, 96, 196])),
    'wc4' : tf.Variable(tf.truncated_normal([1, 1, 196, 196])),
    'wc5' : tf.Variable(tf.truncated_normal([3, 3, 196, 192])),
    'wc6' : tf.Variable(tf.truncated_normal([1, 1, 192, 64])),
    'wc7' : tf.Variable(tf.truncated_normal([1, 1, 192, 10])),
    'wc8' : tf.Variable(tf.truncated_normal([1, 1, 10, 3])),
    }

In [ ]:
cifar_bias = {
    'bc1' : tf.Variable(tf.zeros([96])),
    'bc2' : tf.Variable(tf.zeros([96])),
    'bc3' : tf.Variable(tf.zeros([196])),
    'bc4' : tf.Variable(tf.zeros([196])),
    'bc5' : tf.Variable(tf.zeros([192])),
    'bc6' : tf.Variable(tf.zeros([64])),
    'bc7' : tf.Variable(tf.zeros([10])), 
    'bc8' : tf.Variable(tf.zeros([3])), 
    }

In [22]:
def conv(image, weight_id, bais_id):
    conv = tf.nn.conv2d(image, cifar_weights[weight_id], [1, 1, 1, 1], padding='SAME')
    conv = tf.nn.bias_add(conv, cifar_bias[bais_id])
    return tf.nn.relu(conv)

def max_pool(image, k = 2, stride = 2):
    return tf.nn.max_pool(image, [1, k, k, 1], strides=[1, stride, stride, 1], padding='SAME')

In [ ]:
# Recreating the cifar conv net
def conv_cifar(input_image):
    conv1 = conv(input_image, 'wc1', 'bc1')
#     conv2 = conv(conv1, 'wc2', 'bc2')
    max1 = max_pool(conv1)#conv2
    conv3 = conv(max1, 'wc3', 'bc3')
#     conv4 = conv(conv3, 'wc4', 'bc4')
    max2 = max_pool(conv3)#conv4
    conv5 = conv(max2, 'wc5', 'bc5')
    conv6 = conv(conv5, 'wc6', 'bc6')
    b, h, w, c = conv6.get_shape().as_list()
#     conv7 = conv(conv6, 'wc7', 'bc7')
#     conv8 = conv(conv7, 'wc8', 'bc8')
#     out = tf.nn.avg_pool(conv8, [1, 8, 8, 1], strides=[1, 1, 1, 1], padding='VALID')
#     out = tf.reshape(out, [-1, 3])
    unrolled = tf.reshape(conv6, [-1, h * w * c])
    full1 = tf.add(tf.matmul(unrolled, weights['wf1']), bias['bf1'])
    full1 = tf.nn.relu(full1)
    full2 = tf.add(tf.matmul(full1, weights['wf2']), bias['bf2'])
    full2 = tf.nn.relu(full2)
    out = tf.add(tf.matmul(full2, weights['out']), bias['out'])
    return out

In [ ]:
conv_cifar(tf.random_normal([6, 32, 32, 3]))

In [24]:
conv_net(tf.random_normal([6, 32, 32, 3])) #run excatly twice

<tf.Tensor 'Add_11:0' shape=(6, 3) dtype=float32>

In [25]:
pred = conv_net(X)
# pred = conv_cifar(X)
output = tf.argmax(pred, axis = 1, name = 'final_output')
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=pred))
train_op = tf.train.AdamOptimizer().minimize(loss)
corrected_pred = tf.equal(tf.arg_max(pred, 1), tf.arg_max(y, 1))
acc = tf.reduce_mean(tf.cast(corrected_pred, tf.float32))

In [26]:
saver = tf.train.Saver()

In [27]:
loss_list = []
with tf.Session() as sess:
    
    sess.run(tf.global_variables_initializer())
    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(coord=coord)
    if isfile('models/new_model.ckpt.index'):
        saver.restore(sess, 'models/new_model.ckpt')
    val_img, val_lab = sess.run(vali_batch)
    val_lab = ohe.transform(val_lab.reshape(-1, 1)).toarray()
    print(val_lab.shape)
    for i in range(training_iters):
        image, label = sess.run(train_batch)
        label = ohe.transform(label.reshape(-1, 1)).toarray()
        temp, _ = sess.run([loss, train_op], feed_dict={X:image, y:label, keep_prob:drop})
        if i % display == 0:
            val_loss, accuracy = sess.run([loss, acc], feed_dict={X:val_img, y:val_lab, keep_prob:1.0})
            print('Iter %s, Validation Acc %.4f, Training loss %.4f'%(i, accuracy, val_loss))  
            save_path = saver.save(sess, "models/new_model.ckpt")
    print('Saved and Optimized')
    coord.request_stop()
    coord.join(threads)    

(175, 3)
Iter 0, Validation Acc 0.6857, Training loss 12297.1631
Iter 10, Validation Acc 0.6571, Training loss 10976.6914
Iter 20, Validation Acc 0.6400, Training loss 11640.8350
Iter 30, Validation Acc 0.6629, Training loss 12292.6484
Iter 40, Validation Acc 0.6857, Training loss 12202.7119
Iter 50, Validation Acc 0.6800, Training loss 11365.2598
Iter 60, Validation Acc 0.6686, Training loss 11117.3174
Iter 70, Validation Acc 0.6800, Training loss 11264.8604
Iter 80, Validation Acc 0.6629, Training loss 11387.2275
Iter 90, Validation Acc 0.6800, Training loss 11246.0674
Iter 100, Validation Acc 0.6800, Training loss 11762.5576
Iter 110, Validation Acc 0.6686, Training loss 11403.9668
Iter 120, Validation Acc 0.6686, Training loss 13523.2461
Iter 130, Validation Acc 0.6457, Training loss 11733.2383
Iter 140, Validation Acc 0.6743, Training loss 10857.7734
Iter 150, Validation Acc 0.6914, Training loss 11296.2441
Iter 160, Validation Acc 0.7029, Training loss 11203.2988
Iter 170, Valida

KeyboardInterrupt: 

In [ ]:
freeze_graph('models/')